In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_general6_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_six_month'].values

In [4]:
#### CART
depth = [7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [5]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.5553895279498327, 0.5525064440426269)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_six_month'].values

## columns 
cols = KY_X.columns

In [7]:
c_grid = {'C': [0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [8]:
stumps_summary

{'best_auc': 0.7726706842853237,
 'best_params': {'C': 0.002},
 'number of coefs': 30,
 'dictionary': {'age_at_current_charge19.0': 0.01855399938125272,
  'age_at_current_charge21.0': 0.03436016816662916,
  'age_at_current_charge25.0': 0.0008085673947299758,
  'age_at_current_charge30.0': 0.00567590293879957,
  'age_at_current_charge31.0': 0.01308130448142195,
  'age_at_current_charge35.0': 0.04169674151246061,
  'age_at_current_charge39.0': 0.037889995149749556,
  'age_at_current_charge41.0': 0.0344945176212995,
  'age_at_current_charge44.0': 0.004314464206539161,
  'age_at_current_charge70.0': -0.3633915930203929,
  'p_arrest1': -0.2883017030986704,
  'p_arrest2': 0.7164749439001092,
  'p_arrest3': 0.39017582756131997,
  'p_arrest4': 0.23738343078830243,
  'p_arrest5': 0.07809402418117978,
  'p_arrest6': 0.22928439632808797,
  'p_arrest7': 0.08591286120000424,
  'p_charges1': -0.2669611597861419,
  'p_violence1': 0.20550318142626858,
  'p_felony1': 0.6103444618918926,
  'p_felony2': 

### RiskSLIM

In [9]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_six_month'].values

## columns 
cols = KY_X.columns

In [10]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.0008, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

15

In [11]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [12]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000, 
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/22/19 @ 11:46 AM | 203 rows in lookup table
10/22/19 @ 11:46 AM | ------------------------------------------------------------
10/22/19 @ 11:46 AM | runnning initialization procedure
10/22/19 @ 11:46 AM | ------------------------------------------------------------
10/22/19 @ 11:46 AM | CPA produced 2 cuts
10/22/19 @ 11:46 AM | running naive rounding on 5 solutions
10/22/19 @ 11:46 AM | best objective value: 0.5880
10/22/19 @ 11:46 AM | rounding produced 5 integer solutions
10/22/19 @ 11:46 AM | best objective value is 0.5731
10/22/19 @ 11:46 AM | running sequential rounding on 5 solutions
10/22/19 @ 11:46 AM | best objective value: 0.5880
10/22/19 @ 11:46 AM | sequential rounding produced 1 integer solutions
10/22/19 @ 11:46 AM | best objective value: 0.4234
10/22/19 @ 11:46 AM | polishing 6 solutions
10/22/19 @ 11:46 AM | best objective value: 0.4234
10/22/19 @ 11:46 AM | polishing produced 5 integer solutions
10/22/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:46 AM | adding 219 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3422.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

setting c0 = 0.0 to ensure that intercept is not penalized
10/22/19 @ 11:46 AM | 203 rows in lookup table
10/22/19 @ 11:46 AM | ------------------------------------------------------------
10/22/19 @ 11:46 AM | runnning initialization procedure
10/22/19 @ 11:46 AM | ------------------------------------------------------------
10/22/19 @ 11:46 AM | CPA produced 2 cuts
10/22/19 @ 11:46 AM | running naive rounding on 5 solutions
10/22/19 @ 11:46 AM | best objective value: 0.5857
10/22/19 @ 11:46 AM | rounding produced 5 integer solutions
10/22/19 @ 11:46 AM | best objective value is 0.5721
10/22/19 @ 11:46 AM | running sequential rounding on 5 solutions
10/22/19 @ 11:46 AM | best objective value: 0.5857
10/22/19 @ 11:46 AM | sequential rounding produced 1 integer solutions
10/22/19 @ 11:46 AM | best objective value: 0.4215
10/22/19 @ 11:46 AM | polishing 6 solutions
10/22/19 @ 11:46 AM | best objective value: 0.4215
10/22/19 @ 11:46 AM | polishing produced 5 integer solutions
10/22/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:46 AM | adding 214 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3357.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:47 AM | adding 211 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3407.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:47 AM | adding 220 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3400.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:48 AM | adding 223 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3364.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 11:49 AM | adding 234 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3406.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [14]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.5413457469318662, 0.7435229121674019)

In [15]:
#### save results
summary_general6_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}
%store summary_general6_ky_inter_model

Stored 'summary_general6_ky_inter_model' (dict)


In [17]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM"],
           ["General",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)